In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [84]:
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

## Generando la Base principal
* Se usara la base de **cesados** y la base **Planilla**, esta sera filtrada segun las condiciones de la conclusion
* En esta nueva base se empezara a agregar el resto de informacion para el proyecto

In [86]:
# Extrae Cesados y planilla
cesados = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
cesados = pd.read_excel(
                cesados,
                usecols='E,I,O,P,S,T,W,X'
            ).rename(
                columns = {
                    'Total o Temprana':'TIEMPO_CESE',
                    'Producto':'PRODUCTO',
                    'TIPO':'TIPO_CESE',
                    'MOTIVO PRIMARIO':'MOTIVO_CESE'}
            )

planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
planilla = pd.read_excel(
                planilla,
                usecols='A,B,D:H'
            ).rename(
                columns={
                    'COD':'CODIGO',
                    'REGION/DPTO':'REGION',
                    'DIVISION/GC':'DIVISION'    }
            )

* Al aprecer hay colaboradores que cesaron mas de dos veces, es decir que ingresaron, cesaron, volvieron a ingresar y volvieron a cesar
* Se tomara en cuenta este numero de ceses como una caracteristca (aunque son pocas las personas) ***Revisar**
* Eliminare los colaboradores duplicados considerando solo los ultimos ceses, pues son de los que se tendria data mas reciente
* Genero la columna antiguedad definida como CESE menos INGRESO en meses

In [87]:
# Colaboradores con mas de 2 ceses
cesados.groupby(by = 'CODIGO', as_index=False).agg(n = ('CODIGO','count')).query('n>=2')

,CODIGO,n
425,A22AAAAA34,2
535,A33AAAA149,2
580,A33AAAA21A,2
642,A34AAAAA55,2
688,A36AAAAA18,2
1151,A4AAAAA57A,2
1212,A4AAAAA742,2
1397,A57AAAAA17,2
1408,A57AAAAA37,2
1938,A84AAAAA56,2


In [88]:
cesados = cesados.sort_values(
    by = 'CESE',
    ascending = False # de mayor a menor
).drop_duplicates(
    'CODIGO',
    keep = 'first' # los mas recientes
).iloc[:, [0,2,3,6,7]] #solo selecciono las columnas que usare

In [89]:
cesados['antiguedad_meses'] = (((cesados['CESE'] - cesados['INGRESO']).dt.days)/30).round(0) # Creo antiguedad

* Utilizo como base Planilla que tiene un registro de colabores cesados y activos
* Elimino los duplicados pues, la planilla cuenta con registro de colaboradores por meses, me quedo solo con los mas recientes
* Realizo los filtros para generar la base que quiero

In [90]:
planilla = planilla.drop_duplicates(
    'CODIGO',
    keep = 'last').iloc[:, 1:6] # no me interesa la fecha, solo los valores unicos del codigo

In [91]:
df = pd.merge(
    planilla,
    cesados,
    on = 'CODIGO',
    how = 'left'
).query('PRODUCTO == "GRUPAL"')

* Genero una base para Grupal y Otra para invidual porque seran dos diferentes proyectos con caracteristicas diferentes
* Empiezo a filtrar, Cargo, Tipo de cese, motivo y antiguedad
    * Como uni la base de ceses con planilla se han generado vacios de los colaboradores activos, para ellos debo de tener cuidado en los filtros

In [92]:
df = df[
    (df['CARGO'].isin(['VENTA GRUPAL']))
    & (df['TIPO_CESE'] != 'INVOLUNTARIA') #vacio o voluntaria
    & (~df['MOTIVO_CESE'].isin(['Personal Eventual', 'No le Gusto el Trabajo'])) # los otros motivos o vacios
    & (df['antiguedad_meses'] >= 9) # quito los mayores a 9 meses
    | (df['antiguedad_meses'].isna())] # o que sean activos

* Creo una nueva columna que clasifique si es activo o cesado, usando el TIPO_CESE como referencia
* Reemplazo la fecha de los que no cesaron con la fecha de cierre, es decir el 01/01/2022
* Selecciono solo las variables que usare:
    * Ya no usare INGRESO porque necesito extraerno de otra base donde tenga el ingreso de los activos y cesados
    * Ya no usare TIPO_CESE,  MOTIVO_CESE ya no competen en el analisis
    * Ya no usare antiguedad_meses, pues generare una nueva

In [93]:
df.loc[df['TIPO_CESE'] == 'VOLUNTARIA', 'Estado'] = 'CESADO'
df.loc[df['TIPO_CESE'] != 'VOLUNTARIA', 'Estado'] = 'ACTIVO'
df.loc[df['CESE'].isna(), 'CESE'] = '2022-01-01'
df = df.iloc[:,[0,1,2,3,4,6,10]]

* Esta es la base que usare como principal donde ire agregando la informacion de otras variables

In [94]:
df.sample(5)

,CODIGO,CARGO,AGENCIA,PRODUCTO,REGION,CESE,Estado
1906,1A4AAAAA13,VENTA GRUPAL,OFI IQUITOS,GRUPAL,TERR ORIENTE,2020-09-30,CESADO
2278,A4AAAAA488,VENTA GRUPAL,OFI VIRU,GRUPAL,TERR LA LIBERTAD,2021-04-30,CESADO
2651,A54AAAAA33,VENTA GRUPAL,OFI HUANCAYO 2,GRUPAL,TERR CENTRO,2021-10-18,CESADO
2719,111AAAAA49,VENTA GRUPAL,OFI HUANUCO,GRUPAL,TERR CENTRO,2021-11-03,CESADO
5216,A64AAAAA25,VENTA GRUPAL,OFI PISCO,GRUPAL,TERR SUR CHICO,2022-01-01,ACTIVO


In [95]:
df.isna().sum() #No tengo NAS hasta el mento

CODIGO      0
CARGO       0
AGENCIA     0
PRODUCTO    0
REGION      0
CESE        0
Estado      0
dtype: int64

* Es necesario ver si la base con la que vamos a continuar esta balanceada
    *  Como se muestra tenemos **30% de Cesados y 70% de Activos**, por lo que podemos continuar con el analisis

In [96]:
df.groupby(
    by = 'Estado',
    as_index = False
).agg(
    n = ('CODIGO' , 'count')
)['n'] / df.shape[0]

0    0.693061
1    0.306939
Name: n, dtype: float64

## Agregando variables a la base principal

### Datos personales
* Una tabla con caracteristicias principales e importantes de los colaboradores
    * Como se vio en los casos anteriores hay colaboradores con varias fechas de ingreso, es necesario hacer una limpieza de esto considerando solo los ultimos ingresos
    * Cambio a inicio de mes la fecha de ingreso para tener un estandar

In [16]:
datos_personales = extrae_drive('https://docs.google.com/spreadsheets/d/1ku5CfOhQBpML0gXkuRIIEDjqG0liPdKt/edit#gid=254224400')
datos_personales = pd.read_excel(
                        datos_personales,
                        usecols = 'A, C:G'
                    ).sort_values(
                        by = 'INGRESO',
                        ascending = False
                    ).drop_duplicates(
                        'CODIGO',
                        keep = 'first')

## Obteniendo la fecha inicial del mes en el ingreso    
datos_personales['INGRESO'] = datos_personales['INGRESO'].to_numpy().astype('datetime64[M]')

* Uno la tabla de los datos personales con el dataframe que se tiene hasta el momento
* Hallo la antiguedad con el nuevo cruce y la edad en el cese


In [97]:
df = pd.merge(
        df,
        datos_personales,
        on = 'CODIGO',
        how = 'left')

In [98]:
# Hallando la antiguedad en la Fecha de cese y fecha ingreso
df['antiguedad'] = (((df['CESE'] - df['INGRESO']).dt.days)/30).round(0) #meses
df['edad'] = (((df['CESE'] - df['NACIMIENTO']).dt.days)/360).round(0) #anios

* Encontre negativos en la diferencia de edad, esto debido a que hay personas que tienen una fecha de cese que es de su primer ingreso, y una fecha de segundo ingreso, pero a la fecha no cesan por segunda vez. Para solucionar esto, debo reemplazar la fecha cese por la fecha de cierre que estoy tomando (enero 2021) y pasar de Estado CESADO a ACTIVO.
* Recalculo la columna antiguedad  para tener las nuevas antiguedades con los cambios de fecha


In [99]:
df.query('antiguedad < 0').sort_values(by = 'antiguedad')#['CODIGO'].unique()

,CODIGO,CARGO,AGENCIA,PRODUCTO,REGION,CESE,Estado,INGRESO,NACIMIENTO,SEXO,Educacion,EC,antiguedad,edad
117,A33AAAA163,VENTA GRUPAL,OFI CUSCO,GRUPAL,TERR SUR 2,2019-01-19,CESADO,2020-06-01,1993-11-21,F,GRADO DE BACHILLER,SOLTERO/A,-17.0,26.0
1673,AA8AAA2878,VENTA GRUPAL,OFI HUAYCAN,GRUPAL,TERR LIMA ESTE,2017-10-02,CESADO,2019-03-01,1994-10-07,F,TITULADO,CONCUBINATO / CONVIVIENTE,-17.0,23.0
2171,A11AAAAAA3,VENTA GRUPAL,OFI VENTANILLA,GRUPAL,TERR NORTE CHICO,2019-02-12,CESADO,2020-03-01,1993-08-20,F,EDUCACIÓN UNIVERSITARIA INCOMPLETA,SOLTERO/A,-13.0,26.0
1419,A4AAAAA135,VENTA GRUPAL,OFI PIURA,GRUPAL,TERR PIURA,2017-06-01,CESADO,2018-06-01,1993-04-01,F,GRADO DE BACHILLER,SOLTERO/A,-12.0,25.0
1699,AA7AAA5182,VENTA GRUPAL,OFI SANTA ROSA,GRUPAL,TERR SUR 2,2018-04-17,CESADO,2019-04-01,1992-08-17,F,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,-12.0,26.0
2114,AA7AAA3443,VENTA GRUPAL,OFI CHINCHA,GRUPAL,TERR SUR CHICO,2018-12-31,CESADO,2020-01-01,1992-01-10,F,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,-12.0,27.0
2170,A68AAAAA12,VENTA GRUPAL,OFI CASTILLA,GRUPAL,TERR PIURA,2019-03-31,CESADO,2020-03-01,1995-03-22,F,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,-11.0,24.0


In [100]:
cod = df.query('antiguedad < 0')['CODIGO'].unique()
df.loc[df['CODIGO'].isin(cod), 'CESE'] = '2021-01-01'
df.loc[df['CODIGO'].isin(cod), 'Estado'] = 'ACTIVO'
df['antiguedad'] = (((df['CESE'] - df['INGRESO']).dt.days)/30).round(0) #meses

### Bonos de Venta Grupal
* Este valor diferencia el ingreso extra que tiene un colaborador de otro respecto a su desempeno, lo llamaremos RV por Remuneracion Variable
* Tomare los valores promedio que genero un colaborador desde que esta en la empresa, pero con unas consideraciones
    * Es importante tomar en cuenta el periodo covid donde las ventas y por lo tanto la remuneracion variable es atipica


In [102]:
bonos_venta = extrae_drive('https://docs.google.com/spreadsheets/d/1POTE8j5SneZ0QSvmbtKL_t4hu2odWlTM/edit#gid=578896078')
bonos_venta = pd.read_excel(
                    bonos_venta,
                    usecols = 'A, B, CE'
                ).rename(
                    columns = {
                    'Codigo Trabajador': 'CODIGO',
                    'RV TOTAL': 'RV'}
                )

* Como se ve, los periodos de marzo 2020 a octubre 2020 (inclusive) tienen valores muy bajos debido a la pandemia, por lo que para este analisis no lo tomaremos en cuenta, pues nos bajaria el promedio atipicamente

In [103]:
rv_agrup = bonos_venta.groupby(
                by = 'MES',
                as_index = False
            ).agg(
                rv_total = ('RV' , 'sum')
            )

principal = alt.Chart(
    rv_agrup
).mark_line(
    color = '#04328C'
).encode(
    x = alt.X(
        'MES:T',
        title = None),
    y = alt.Y(
        'rv_total:Q',
        title = None,
        axis = None)
)

linea_i = alt.Chart(
    pd.DataFrame({'MES': ['2020-03-01']})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'MES:T')

linea_f = alt.Chart(
    pd.DataFrame({'MES': ['2020-10-01']})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'MES:T')


(principal + linea_i + linea_f).properties(
    title = 'RV total Historico',
    width = 700,
    height = 300
).configure_view(
    strokeWidth = 0
 ).configure_axis(
    labelFontSize = 13,
    labelAngle = 0
 ).configure_title(
    fontSize = 18,
    font = 'courier',
    color = '#033E8C',
    anchor = 'start'
)

alt.LayerChart(...)

* Creando la nueva base segun los filtro considerados
* Utilizaremos el rv promedio y la media pues estos varian mucho en algunos casos

In [104]:
bonos_venta = bonos_venta.query(
                'MES <= "2020-03-01" | MES >= "2020-10-01" '
            ).groupby(
                by = 'CODIGO',
                as_index = False
            ).agg(
                rv_prom = ('RV', 'mean'),
                rv_median = ('RV', 'median')
            )

In [105]:
df.sample(3)

,CODIGO,CARGO,AGENCIA,PRODUCTO,REGION,CESE,Estado,INGRESO,NACIMIENTO,SEXO,Educacion,EC,antiguedad,edad
740,A49AAAAA93,VENTA GRUPAL,OFI JULIACA,GRUPAL,TERR SUR 2,2021-10-22,CESADO,2020-02-01,1989-09-20,M,EDUCACIÓN UNIVERSITARIA INCOMPLETA,SOLTERO/A,21.0,33.0
805,AA8AAA1732,GRCG,OFI TARAPOTO,GRUPAL,TERR ORIENTE,2022-01-01,ACTIVO,2013-12-01,1989-12-05,M,EDUCACIÓN UNIVERSITARIA COMPLETA,CASADO,98.0,33.0
2178,A68AAAAA41,VENTA GRUPAL,OFI LA UNION,GRUPAL,TERR PIURA,2022-01-01,ACTIVO,2020-03-01,1994-11-30,M,TITULADO,CONCUBINATO / CONVIVIENTE,22.0,27.0


In [77]:
df = pd.merge(
        df,
        bonos_venta,
        on = 'CODIGO',
        how = 'left')

In [82]:
df[df['rv_prom_x'].isnull()]

,CODIGO,CARGO,AGENCIA,PRODUCTO,REGION,CESE,Estado,INGRESO,NACIMIENTO,SEXO,Educacion,EC,antiguedad,edad,rv_prom_x,rv_median_x,rv_prom_y,rv_median_y
219,AA8AAA1758,GDCG,OFI CHICLAYO,GRUPAL,TERR LAMBAYEQUE,2022-01-01,ACTIVO,2013-12-01,1977-09-12,M,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,98.0,45.0,NaN,NaN,NaN,NaN
333,A16AAAAA28,JCG,OFI COLLIQUE,GRUPAL,TERR LIMA NORTE,2022-01-01,ACTIVO,2018-07-01,1993-09-30,M,GRADO DE BACHILLER,SOLTERO/A,43.0,29.0,NaN,NaN,NaN,NaN
449,A4AAAAA2A8,JCG,OFI SULLANA,GRUPAL,TERR NORTE 2,2022-01-01,ACTIVO,2016-02-01,1995-07-18,M,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,72.0,27.0,NaN,NaN,NaN,NaN
542,A17AAAAA69,JCG,OFI CERES,GRUPAL,TERR LIMA ESTE,2022-01-01,ACTIVO,2019-11-01,1986-12-12,M,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,26.0,36.0,NaN,NaN,NaN,NaN
699,AA8AAA1716,JCG,OFI CHICLAYO,GRUPAL,TERR LAMBAYEQUE,2022-01-01,ACTIVO,2013-12-01,1984-11-20,F,EDUCACIÓN UNIVERSITARIA COMPLETA,SOLTERO/A,98.0,38.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,A35AAAAA72,VENTA GRUPAL,OFI HUACHIPA,GRUPAL,TERR LIMA ESTE,2022-01-01,ACTIVO,2021-12-01,1995-05-28,M,EDUCACIÓN SECUNDARIA COMPLETA,SOLTERO/A,1.0,27.0,NaN,NaN,NaN,NaN
2445,A99AAAAA47,VENTA GRUPAL,OFI VILLA MARIA DEL TRIUNFO,GRUPAL,TERR LIMA SUR,2022-01-01,ACTIVO,2021-12-01,1990-08-08,F,"EDUCACIÓN SUPERIOR (INSTITUTO SUPERIOR, ETC) C...",SOLTERO/A,1.0,32.0,NaN,NaN,NaN,NaN
2446,A49AAAA116,VENTA GRUPAL,OFI JULIACA,GRUPAL,TERR SUR 2,2022-01-01,ACTIVO,2021-12-01,1991-11-13,F,EDUCACIÓN UNIVERSITARIA COMPLETA,CONCUBINATO / CONVIVIENTE,1.0,31.0,NaN,NaN,NaN,NaN
2447,A22AAAAA77,VENTA GRUPAL,OFI PAUCARPATA,GRUPAL,TERR SUR 1,2022-01-01,ACTIVO,2021-12-01,1999-05-03,F,"EDUCACIÓN SUPERIOR (INSTITUTO SUPERIOR, ETC) C...",SOLTERO/A,1.0,23.0,NaN,NaN,NaN,NaN


In [79]:
df.isna().sum()

CODIGO           0
CARGO            0
AGENCIA          0
PRODUCTO         0
REGION           0
CESE             0
Estado           0
INGRESO          0
NACIMIENTO       0
SEXO             0
Educacion        0
EC               0
antiguedad       0
edad             0
rv_prom_x      229
rv_median_x    229
rv_prom_y      229
rv_median_y    229
dtype: int64



programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')
vacaciones = extrae_drive('https://docs.google.com/spreadsheets/d/1riJOPN5KUwV1962wO6mjPL7PMgL2v3_s/edit#gid=800276956')
descanso_medico = extrae_drive('https://docs.google.com/spreadsheets/d/1H_oENDjS0FyDNVmSg1199PrE6s8q6z7-/edit#gid=1138295584')
bonos_venta_i = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')

desempeno = extrae_drive()
reconocimientos = extrae_drive()
hijxs = extrae_drive()

recategorizaciones = extrae_drive()
cobertura_ofi = extrae_drive()
antg_oficina = extrae_drive()
gptw_oficina = extrae_drive()